In [1]:
import numpy as np
import pandas as pd
import scipy.stats

In [3]:
f1 = pd.read_excel('Data/Data_ELISA_刪減版.xlsx')
f1.head(10)

,group 5,收案院區,收案年份,性別,Age,有無喝酒(A),有無嚼檳榔(B),有無抽菸(C),A+B+C,Stage,T,N,M,Defferentiation_G,Site,Marker_Conc.
0,1,1,1,1,41.849315,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1,1,1,1,47.065753,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1,2,1,1,54.989041,0.0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1,2,1,1,53.824658,0.0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,1,1,1,1,57.238356,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,1,1,1,1,61.139726,0.0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,1,1,1,1,56.271233,0.0,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,1,1,1,1,49.136986,1.0,1,1,3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,1,1,1,1,41.054795,1.0,1,1,3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,1,1,1,1,65.391781,0.0,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [4]:
#看各個column的內容
colnum = list(f1.columns[1:-1])
colnum.remove('Age')
for i in colnum:
    ls = f1[i]
    ls = [str(x) for x in ls]
    print("%s:"%i,set(ls))
    
#'2 or NA' --> 看成Missing data

收案院區: {'1', '2'}
收案年份: {'3', '1', '4', '2'}
性別: {'1', '2'}
有無喝酒(A): {'nan', '1.0', '0.0'}
有無嚼檳榔(B): {'0', '1'}
有無抽菸(C): {'0', '1'}
A+B+C: {'1', '0', '2 or NA', '2', '3'}
Stage: {'nan', '2.0', '1.0', '4.0', '3.0', '0.0'}
T: {'nan', '2.0', '1.0', '4.0', '3.0', '0.0'}
N: {'nan', '2.0', '1.0', '3.0', '0.0'}
M: {'nan', '1.0', '0.0'}
Defferentiation_G: {'nan', '2.0', '1.0', '3.0', '0.0'}
Site: {'nan', '2.0', '5.0', '1.0', '7.0', '4.0', '3.0', '6.0'}


In [6]:
# 個數統計-------------------------------------------------------------
df_data = []

for gp in sorted(list(set(f1['group 5']))):
    ls = []

    #Total case number
    ls.append(sum(f1['group 5']==gp))

    #Year (2011-2012/2013/2014/2015-2016)
    yr_number = []
    for yr in sorted(list(set(f1['收案年份']))):
        yr_number.append(str(len(f1[(f1['group 5']==gp)&(f1['收案年份']==yr)])))
    ls.append('/'.join(yr_number))

    #Hospital (Chi-Mei/CGMF-KH)
    hs_number = []
    for hs in sorted(list(set(f1['收案院區']))):
        hs_number.append(str(len(f1[(f1['group 5']==gp)&(f1['收案院區']==hs)])))
    ls.append('/'.join(hs_number))

    #Number of male/female
    gender_number = []
    for g in sorted(list(set(f1['性別']))):
        gender_number.append(str(len(f1[(f1['group 5']==gp)&(f1['性別']==g)])))
    ls.append('/'.join(gender_number))

    #Range/median of age (y)
    age_list = f1[f1['group 5']==gp]['Age']
    min_age = int(min(age_list))
    max_age = int(max(age_list))
    med_age = int(np.median(age_list))
    ls.append(str(min_age)+'-'+str(max_age)+"/"+str(med_age))

    #No. of individuals with habitual behaviors
    ls.append('')
    ls.append(len(f1[(f1['group 5']==gp) & (f1['有無喝酒(A)']==1)]))
    ls.append(len(f1[(f1['group 5']==gp) & (f1['有無嚼檳榔(B)']==1)]))
    ls.append(len(f1[(f1['group 5']==gp) & (f1['有無抽菸(C)']==1)]))
    abc_number = []
    for abc in [0,1,2,3]:
        abc_number.append(str(len(f1[(f1['group 5']==gp) & (f1['A+B+C']==abc)])))
    ls.append('/'.join(abc_number))

    #Site
    ls.append('')
    site_list = [1,2,3,4,5,6,7]
    for s in site_list:
        site_len = len(f1[(f1['group 5']==gp) & (f1['Site']==s)])
        if site_len == 0:
            ls.append('-')
        else:
            ls.append(site_len)
    ls.append(len(f1[(f1['group 5']==gp) & (pd.core.series.Series(map(str,f1['Site']))=='nan')]))

    #No. of patients with non/well/moderate/poor-differentiated cancer
    diff_list = [0,1,2,3]
    diff_number = []
    for d in diff_list:
        diff_number.append(str(len(f1[(f1['group 5']==gp) & (f1['Defferentiation_G']==d)])))
    diff_str = '/'.join(diff_number)
    if diff_str == '0/0/0/0':
        ls.append('-')
    else:
        na_number = len(f1[(f1['group 5']==gp) & (pd.core.series.Series(map(str,f1['Defferentiation_G']))=='nan')])
        ls.append('%s (na=%s)'%(diff_str,na_number))

    #No. of patients with T0/T1/T2/T3/T4 pT status
    pT_list = [0,1,2,3,4]
    pT_number = []
    for t in pT_list:
        pT_number.append(str(len(f1[(f1['group 5']==gp) & (f1['T']==t)])))
    pT_str = '/'.join(pT_number)
    if pT_str == '0/0/0/0/0':
        ls.append('-')
    else:
        na_number = len(f1[(f1['group 5']==gp) & (pd.core.series.Series(map(str,f1['T']))=='nan')])
        ls.append('%s (na=%s)'%(pT_str,na_number))

    #No. of patients with N0/N1/N2/N3 pN status
    pN_list = [0,1,2,3]
    pN_number = []
    for n in pN_list:
        pN_number.append(str(len(f1[(f1['group 5']==gp) & (f1['N']==n)])))
    pN_str = '/'.join(pN_number)
    if pN_str == '0/0/0/0':
        ls.append('-')
    else:
        na_number = len(f1[(f1['group 5']==gp) & (pd.core.series.Series(map(str,f1['N']))=='nan')])
        ls.append('%s (na=%s)'%(pN_str,na_number))

    #No. of patients with overall stage 0/I/II/III/IV
    Stage_list = [0,1,2,3,4]
    Stage_number = []
    for s in Stage_list:
        Stage_number.append(str(len(f1[(f1['group 5']==gp) & (f1['Stage']==s)])))
    Stage_str = '/'.join(Stage_number)
    if Stage_str == '0/0/0/0/0':
        ls.append('-')
    else:
        Stage_number = len(f1[(f1['group 5']==gp) & (pd.core.series.Series(map(str,f1['Stage']))=='nan')])
        ls.append('%s (na=%s)'%(Stage_str,na_number))
    df_data.append(ls)
df_data = np.asmatrix(df_data)
outputdata = pd.DataFrame(df_data.T,columns=['Healthy control','LR-OPMD','HR-OPMD','OSCC','Outside oral cavity(16cancer)'])

df_index = []
df_index.append('Total case number')
df_index.append('Year (2011-2012/2013/2014/2015-2016)')
df_index.append('Hospital (Chi-Mei/CGMF-KH)')
df_index.append('Number of male/female')
df_index.append('Range/median of age (y)')
df_index.append('No. of individuals with habitual behaviors')
df_index.append('Alcohol consumption (A)')
df_index.append('Betel nut chewing (B)')
df_index.append('Smoking (C)')
df_index.append('All-/one+/two+/all+ with ABC')
df_index.append('Site')
df_index.append('Buccal mucosa (include Retromolar trigone)')
df_index.append('Gingiva (Gum)')
df_index.append('Tongue')
df_index.append('Palate (include Uvula)')
df_index.append('Other (Mouth floor、Lip、Tonsil)')
df_index.append('>2 site')
df_index.append('outside oral cavity')
df_index.append('NA')
df_index.append('No. of patients with non/well/moderate/poor-differentiated cancer')
df_index.append('No. of patients with T0/T1/T2/T3/T4 pT status')
df_index.append('No. of patients with N0/N1/N2/N3 pN status')
df_index.append('No. of patients with overall stage 0/I/II/III/IV')

df_index = dict(zip(list(range(0,23)),df_index))

outputdata = outputdata.rename(df_index)
outputdata.to_excel('output/20190923-table.xlsx')

In [7]:
# 統計檢定-------------------------------------------------------------

df_data=[]

for gp in sorted(list(set(f1['group 5']))):
    CaseNumber = []
    Mean_sd = []
    p_value = []

    #Year 1:2011-2012 2:2013 3:2014 4:2015-2016
    sample1 = f1[(f1['group 5']==gp) & (f1['收案年份']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['收案年份']==2)]
    sample3 = f1[(f1['group 5']==gp) & (f1['收案年份']==3)]
    sample4 = f1[(f1['group 5']==gp) & (f1['收案年份']==4)]
    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    x3 = list(sample3['Marker_Conc.'])
    x4 = list(sample4['Marker_Conc.'])
    res = scipy.stats.kruskal(x1,x2,x3,x4)
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x3))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x3),np.std(x3,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x4))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x4),np.std(x4,ddof = 1)))
    p_value.append('')


    #Hospital 1:奇美 2:高長
    sample1 = f1[(f1['group 5']==gp) & (f1['收案院區']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['收案院區']==2)]
    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    res = scipy.stats.mannwhitneyu(x1,x2,alternative='two-sided')
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')

    #Gender 1:male 2:female
    sample1 = f1[(f1['group 5']==gp) & (f1['性別']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['性別']==2)]
    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    res = scipy.stats.mannwhitneyu(x1,x2,alternative='two-sided')
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')

    #Alcohol consumption(A) 1:有 0:無
    sample1 = f1[(f1['group 5']==gp) & (f1['有無喝酒(A)']==1)]
    sample0 = f1[(f1['group 5']==gp) & (f1['有無喝酒(A)']==0)]
    x1 = list(sample1['Marker_Conc.'])
    x0 = list(sample0['Marker_Conc.'])
    res = scipy.stats.mannwhitneyu(x1,x0,alternative='two-sided')
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x0))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x0),np.std(x0,ddof = 1)))
    p_value.append('')

    #Betel nut chewing(B) 1:有 0:無
    sample1 = f1[(f1['group 5']==gp) & (f1['有無嚼檳榔(B)']==1)]
    sample0 = f1[(f1['group 5']==gp) & (f1['有無嚼檳榔(B)']==0)]
    x1 = list(sample1['Marker_Conc.'])
    x0 = list(sample0['Marker_Conc.'])
    res = scipy.stats.mannwhitneyu(x1,x0,alternative='two-sided')
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x0))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x0),np.std(x0,ddof = 1)))
    p_value.append('')

    #Smoking(C) 1:有 0:無
    sample1 = f1[(f1['group 5']==gp) & (f1['有無抽菸(C)']==1)]
    sample0 = f1[(f1['group 5']==gp) & (f1['有無抽菸(C)']==0)]
    x1 = list(sample1['Marker_Conc.'])
    x0 = list(sample0['Marker_Conc.'])
    res = scipy.stats.mannwhitneyu(x1,x0,alternative='two-sided')
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x0))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x0),np.std(x0,ddof = 1)))
    p_value.append('')

    #A+B+C 0:All- 1:one+ 2:two+ 3:all+
    sample0 = f1[(f1['group 5']==gp) & (f1['A+B+C']==0)]
    sample1 = f1[(f1['group 5']==gp) & (f1['A+B+C']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['A+B+C']==2)]
    sample3 = f1[(f1['group 5']==gp) & (f1['A+B+C']==3)]
    x0 = list(sample0['Marker_Conc.'])
    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    x3 = list(sample3['Marker_Conc.'])
    res = scipy.stats.kruskal(x0,x1,x2,x3)
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x0))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x0),np.std(x0,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x3))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x3),np.std(x3,ddof = 1)))
    p_value.append('')

    #Site 
    #1:Buccal mucosa (include Retromolar trigone)
    #2:Gingiva (Gum)
    #3:Tongue
    #4:Palate (include Uvula)
    #5:Other (Mouth floor、Lip、Tonsil)
    #6:>2 site
    #7:outside oral cavity
    sample1 = f1[(f1['group 5']==gp) & (f1['Site']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['Site']==2)]
    sample3 = f1[(f1['group 5']==gp) & (f1['Site']==3)]
    sample4 = f1[(f1['group 5']==gp) & (f1['Site']==4)]
    sample5 = f1[(f1['group 5']==gp) & (f1['Site']==5)]
    sample6 = f1[(f1['group 5']==gp) & (f1['Site']==6)]
    sample7 = f1[(f1['group 5']==gp) & (f1['Site']==7)]

    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    x3 = list(sample3['Marker_Conc.'])
    x4 = list(sample4['Marker_Conc.'])
    x5 = list(sample5['Marker_Conc.'])
    x6 = list(sample6['Marker_Conc.'])
    x7 = list(sample7['Marker_Conc.'])

    res = scipy.stats.kruskal(x1,x2,x3,x4,x5,x6)
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x3))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x3),np.std(x3,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x4))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x4),np.std(x4,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x5))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x5),np.std(x5,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x6))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x6),np.std(x6,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x7))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x7),np.std(x7,ddof = 1)))
    p_value.append('')

    #differentiation of cancer
    #0: non
    #1: well
    #2: moderate
    #3: poor
    sample0 = f1[(f1['group 5']==gp) & (f1['Defferentiation_G']==0)]
    sample1 = f1[(f1['group 5']==gp) & (f1['Defferentiation_G']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['Defferentiation_G']==2)]
    sample3 = f1[(f1['group 5']==gp) & (f1['Defferentiation_G']==3)]
    x0 = list(sample0['Marker_Conc.'])
    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    x3 = list(sample3['Marker_Conc.'])
    res = scipy.stats.kruskal(x0,x1,x2,x3)
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x0))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x0),np.std(x0,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x3))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x3),np.std(x3,ddof = 1)))
    p_value.append('')

    #pT status
    #0: T0
    #1: T1
    #2: T2
    #3: T3
    #4: T4
    sample0 = f1[(f1['group 5']==gp) & (f1['T']==0)]
    sample1 = f1[(f1['group 5']==gp) & (f1['T']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['T']==2)]
    sample3 = f1[(f1['group 5']==gp) & (f1['T']==3)]
    sample4 = f1[(f1['group 5']==gp) & (f1['T']==4)]
    x0 = list(sample0['Marker_Conc.'])
    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    x3 = list(sample3['Marker_Conc.'])
    x4 = list(sample4['Marker_Conc.'])
    res = scipy.stats.kruskal(x0,x1,x2,x3,x4)
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x0))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x0),np.std(x0,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x3))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x3),np.std(x3,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x4))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x4),np.std(x4,ddof = 1)))
    p_value.append('')

    #pN status
    #0: N0
    #1: N1
    #2: N2
    #3: N3
    sample0 = f1[(f1['group 5']==gp) & (f1['N']==0)]
    sample1 = f1[(f1['group 5']==gp) & (f1['N']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['N']==2)]
    sample3 = f1[(f1['group 5']==gp) & (f1['N']==3)]
    x0 = list(sample0['Marker_Conc.'])
    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    x3 = list(sample3['Marker_Conc.'])
    res = scipy.stats.kruskal(x0,x1,x2,x3)
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x0))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x0),np.std(x0,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x3))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x3),np.std(x3,ddof = 1)))
    p_value.append('')

    #Stage
    #0: 0
    #1: I
    #2: II
    #3: III
    #4: IV
    sample0 = f1[(f1['group 5']==gp) & (f1['Stage']==0)]
    sample1 = f1[(f1['group 5']==gp) & (f1['Stage']==1)]
    sample2 = f1[(f1['group 5']==gp) & (f1['Stage']==2)]
    sample3 = f1[(f1['group 5']==gp) & (f1['Stage']==3)]
    sample4 = f1[(f1['group 5']==gp) & (f1['Stage']==4)]
    x0 = list(sample0['Marker_Conc.'])
    x1 = list(sample1['Marker_Conc.'])
    x2 = list(sample2['Marker_Conc.'])
    x3 = list(sample3['Marker_Conc.'])
    x4 = list(sample4['Marker_Conc.'])
    res = scipy.stats.kruskal(x0,x1,x2,x3,x4)
    p = res[1]

    CaseNumber.append('')
    Mean_sd.append('')
    p_value.append('')
    CaseNumber.append(len(x0))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x0),np.std(x0,ddof = 1)))
    p_value.append('%f'%p)
    CaseNumber.append(len(x1))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x1),np.std(x1,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x2))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x2),np.std(x2,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x3))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x3),np.std(x3,ddof = 1)))
    p_value.append('')
    CaseNumber.append(len(x4))
    Mean_sd.append('%.1f±%.1f'%(np.mean(x4),np.std(x4,ddof = 1)))
    p_value.append('')
    
    df_data.append(CaseNumber)
    df_data.append(Mean_sd)
    df_data.append(p_value)
    
df_data = np.asmatrix(df_data)
colname = []
colname.extend(['Case Number of Health Control','Marker Conc.(mean±sd)','p-value'])
colname.extend(['Case Number of LR-OPMD','Marker Conc.(mean±sd)','p-value'])
colname.extend(['Case Number of HR-OPMD','Marker Conc.(mean±sd)','p-value'])
colname.extend(['Case Number of OSCC','Marker Conc.(mean±sd)','p-value'])
colname.extend(['Case Number of Outside Oral Cavity','Marker Conc.(mean±sd)','p-value'])
outputdata = pd.DataFrame(df_data.T,columns=colname)

df_index = []
df_index.extend(['Year','2011-2012','2013','2014','2015-2016'])
df_index.extend(['Hospital','Chi-Mei','CGMF-KH'])
df_index.extend(['Gender','male','female'])
df_index.extend(['Alcohol consumption(A)','1','0'])
df_index.extend(['Betel nut chewing(B)','1','0'])
df_index.extend(['Smoking(C)','1','0'])
df_index.extend(['No. of individuals with habitual behaviors','all-','one+','two+','all+'])
df_index.extend(['Site','Buccal mucosa (include Retromolar trigone)','Gingiva (Gum)','Tongue','Palate (include Uvula)','Other (Mouth floor、Lip、Tonsil)','>2 site','outside oral cavity'])
df_index.extend(['differentiation of cancer','non','well','moderate','poor'])
df_index.extend(['pT status','T0','T1','T2','T3','T4'])
df_index.extend(['pN status','N0','N1','N2','N3'])
df_index.extend(['Stage','0','I','II','III','IV'])

df_index = dict(zip(list(range(0,len(df_index))),df_index))

outputdata = outputdata.rename(df_index)
outputdata.to_excel('output/20190923-statistic_analysis.xlsx')

C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of emp

C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\chienhua\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of emp